In [39]:
import pandas as pd
import numpy as np
import os
import re

# ===================================
# Canonical Countries + Name Mapping
# ===================================
CANON_COUNTRIES = [
    "India", "USA", "Russia", "France", "Germany", "Italy", "China", "Japan",
    "Argentina", "Portugal", "Spain", "Croatia", "Belgium", "Australia",
    "Pakistan", "Afghanistan", "Israel", "Iran", "Iraq", "Bangladesh",
    "Sri Lanka", "Canada", "UK", "Sweden", "Saudi Arabia"
]

COUNTRY_MAP = {
    "United States": "USA", "United States of America": "USA", "U.S.": "USA", "US": "USA",
    "Russian Federation": "Russia",
    "United Kingdom": "UK", "Great Britain": "UK", "Britain": "UK", "U.K.": "UK",
    "United Kingdom of Great Britain and Northern Ireland": "UK",
    "Iran, Islamic Rep.": "Iran", "Islamic Republic of Iran": "Iran", "Iran (Islamic Republic of)": "Iran",
    "Republic of Iraq": "Iraq",
    "Islamic Republic of Afghanistan": "Afghanistan",
    "China, P.R.: Mainland": "China", "People's Republic of China": "China", "Mainland China": "China",
    "Dem. Socialist Republic of Sri Lanka": "Sri Lanka", "SriLanka": "Sri Lanka",
    "Saudi Arabia, Kingdom of": "Saudi Arabia"
}

def standardize_country_names(df, col="Country"):
    if col not in df.columns:
        return df
    df[col] = df[col].astype(str).str.strip().replace(COUNTRY_MAP)
    return df[df[col].isin(CANON_COUNTRIES)].copy()

def coerce_year_to_int64(series):
    s = series.astype(str).str.extract(r'(\d{4})', expand=False)
    return pd.to_numeric(s, errors="coerce").astype("Int64")

def clean_wide_metric_names(cols):
    cleaned = []
    for c in cols:
        c = re.sub(r"\s+", "_", str(c).strip())
        c = c.replace("%", "pct").replace("/", "_per_").replace("(", "").replace(")", "")
        c = re.sub(r"[^0-9a-zA-Z_]+", "_", c)
        c = re.sub(r"_+", "_", c).strip("_")
        cleaned.append(c)
    return cleaned

# ===================================
# Dataset Preprocessors
def drop_sparse_cols(df, min_non_missing=0.2): 
    """
    Drop columns with fewer than `min_non_missing` fraction of non-missing values.
    Example: min_non_missing=0.3 → drop column if <30% non-missing values.
    """
    thresh = int(len(df) * min_non_missing)
    return df.dropna(axis=1, thresh=thresh)


def preprocess_agriculture(filepath, missing_threshold=0.8):
    df = pd.read_csv(filepath)

    # Rename
    df.rename(columns={"Area": "Country"}, inplace=True)

    df.columns = df.columns.str.strip()

    # Convert Year and Value to numeric
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
    df["Value"] = pd.to_numeric(df["Value"], errors="coerce")

    # Replace placeholders with NaN
    df.replace(["..", "", "NaN"], np.nan, inplace=True)

    # Fill missing numeric Value with 0
    df["Value"] = df["Value"].fillna(0)

    # Standardize country names
    df = standardize_country_names(df, "Country")

    # Filter years
    df = df[(df["Year"] >= 2000) & (df["Year"] <= 2024)]

    # Drop columns with >80% missing values (applies to numeric & categorical)
    threshold_count = int(len(df) * missing_threshold)
    df = df.dropna(axis=1, thresh=len(df) - threshold_count)


    return df



def summarize_missing(df, title=""):
    print(f"\n--- {title} ---")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} cols")
    missing_summary = (
        df.isna().sum()
        .to_frame("Missing_Count")
        .assign(Missing_Percent=lambda x: (x["Missing_Count"] / len(df)) * 100)
        .query("Missing_Count > 0")
        .sort_values("Missing_Percent", ascending=False)
    )
    if missing_summary.empty:
        print("No missing values ✅")
    else:
        print(missing_summary)


def preprocess_disasters(filepath, savepath="cleaned_dataset.csv"):
    df = pd.read_csv(filepath)

    # --- Summary BEFORE dropping ---
    summarize_missing(df, "Before Dropping Sparse Columns")

    # --- Standardize Year column ---
    if "Start Year" in df.columns:
        df.rename(columns={"Start Year": "Year"}, inplace=True)
    if "Year" in df.columns:
        df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")

    # --- Replace placeholder missing values with NaN ---
    df.replace(["..", "", "NaN", "nan", "NULL", "N/A"], np.nan, inplace=True)

    # --- Standardize country names ---
    if "Country" in df.columns and "standardize_country_names" in globals():
        df = standardize_country_names(df, "Country")

    # --- Keep only years in valid range ---
    if "Year" in df.columns:
        df = df[(df["Year"] >= 2000) & (df["Year"] <= 2024)]

    # --- Drop sparse cols (>70% missing) BEFORE imputing ---
    thresh = len(df) * 0.2
    df = df.dropna(axis=1, thresh=thresh)

    # --- Summary AFTER dropping ---
    summarize_missing(df, "After Dropping Sparse Columns")

    # --- Handle remaining missing values ---
    for col in df.columns:
        if df[col].isna().sum() > 0:
            if df[col].dtype.kind in "biufc":  # numeric
                df[col] = df[col].fillna(0)   # or mean/median
            else:  # categorical
                df[col] = df[col].fillna("Unknown")

    # --- Drop duplicates ---
    df = df.drop_duplicates()

    # --- Save ---
    df.to_csv(savepath, index=False)
    return df




def preprocess_population(filepath, missing_threshold=0.8):
    df = pd.read_csv(filepath)
    
    # Strip column names
    df.columns = df.columns.str.strip()
    
    # Rename
    df.rename(columns={"Area": "Country"}, inplace=True)
    
    # Convert Year to numeric
    if "Year" in df: 
        df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
    
    # Replace placeholders
    df.replace(["..", "", "NaN"], np.nan, inplace=True)
    
    # Convert Value
    if "Unit" in df and "Value" in df:
        df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
        mask = (df["Unit"] == "1000 No") & df["Value"].notna()
        df.loc[mask, "Value"] *= 1000
    
    # Standardize country names
    df = standardize_country_names(df, "Country")
    
    # Filter years
    df = df[(df["Year"] >= 2000) & (df["Year"] <= 2024)]
    
    # Drop sparse columns (both numeric & categorical)
    threshold_count = int(len(df) * missing_threshold)
    df = df.dropna(axis=1, thresh=len(df) - threshold_count)
    
    # Drop duplicates
    df = df.drop_duplicates()
    
    return df


def preprocess_resilience(filepath):
    df = pd.read_csv(filepath)
    id_cols = [c for c in ["Country Name", "Country Code", "Series Name", "Series Code"] if c in df]
    year_cols = [c for c in df if re.search(r"\b(200\d|201\d|202[0-4])\b", str(c))]
    df = df.dropna(subset=year_cols, how="all")

    df_long = df.melt(id_vars=id_cols, value_vars=year_cols,
                      var_name="Year_raw", value_name="Value")
    df_long["Year"] = coerce_year_to_int64(df_long["Year_raw"])
    df_long.dropna(subset=["Year"], inplace=True)
    df_long["Value"] = pd.to_numeric(df_long["Value"], errors="coerce")
    df_long = df_long[df_long["Year"].between(2000, 2024)]
    if "Country Name" in df_long: df_long.rename(columns={"Country Name": "Country"}, inplace=True)
    df_long = standardize_country_names(df_long, "Country")

    # pivot → wide
    df_pivot = df_long.pivot_table(index=["Country", "Year"],
                                   columns="Series Name",
                                   values="Value",
                                   aggfunc="first")
    df_pivot.columns = clean_wide_metric_names(df_pivot.columns)
    df_pivot.reset_index(inplace=True)

    df_pivot = drop_sparse_cols(df_pivot)
    numeric_cols = df_pivot.select_dtypes(include="number").columns
    for col in numeric_cols:
        df_pivot[col] = (df_pivot.sort_values(["Country", "Year"])
                                  .groupby("Country")[col]
                                  .transform(lambda x: x.interpolate().ffill().bfill()))
    df_pivot[numeric_cols] = df_pivot[numeric_cols].fillna(df_pivot[numeric_cols].median())
    return df_pivot.drop_duplicates().sort_values(["Country", "Year"]).reset_index(drop=True)

def make_unique(df, name):
    if df.duplicated(subset=["Country", "Year"]).any():
        print(f"[INFO] Fixing duplicates in {name}...")
        df = df.groupby(["Country", "Year"], as_index=False).mean(numeric_only=True)
    return df

# ===================================
# Paths
# ===================================
paths = {
    "raw_dir": "../data_raw",
    "clean_dir": "../data_clean",
}

os.makedirs(paths["clean_dir"], exist_ok=True)

# ===================================
# Run Preprocessing
# ===================================
agri_clean = preprocess_agriculture(os.path.join(paths["raw_dir"], "crop_and_livestock.csv"))
population_clean = preprocess_population(os.path.join(paths["raw_dir"], "population_and_demographics.csv"))
disaster_clean = preprocess_disasters(os.path.join(paths["raw_dir"], "disasters.csv"))
resilience_clean = preprocess_resilience(os.path.join(paths["raw_dir"], "Resiliance.csv"))

# Ensure uniqueness
agri_clean = make_unique(agri_clean, "Agriculture")
population_clean = make_unique(population_clean, "Population")
disaster_clean = make_unique(disaster_clean, "Disaster")
resilience_clean = make_unique(resilience_clean, "Resilience")

# ===================================
# Save each cleaned dataset
# ===================================
agri_clean.to_csv(os.path.join(paths["clean_dir"], "agriculture_clean.csv"), index=False)
population_clean.to_csv(os.path.join(paths["clean_dir"], "population_clean.csv"), index=False)
disaster_clean.to_csv(os.path.join(paths["clean_dir"], "disasters_clean.csv"), index=False)
resilience_clean.to_csv(os.path.join(paths["clean_dir"], "resilience_clean.csv"), index=False)

print("✅ Individual cleaned datasets saved!")

# ===================================
# Merge all datasets
# ===================================
merged_df = (
    agri_clean
    .merge(population_clean, on=["Country", "Year"], how="outer")
    .merge(disaster_clean, on=["Country", "Year"], how="outer")
    .merge(resilience_clean, on=["Country", "Year"], how="outer")
)

# Fill NaN in disaster-related columns
disaster_cols = [c for c in merged_df if "Disaster" in c]
merged_df[disaster_cols] = merged_df[disaster_cols].fillna(0)

# Drop remaining nulls safely
merged_df = merged_df.dropna().reset_index(drop=True)

# Save final merged dataset
out_path = os.path.join(paths["clean_dir"], "merged_country_year.csv")
merged_df.to_csv(out_path, index=False)

print("✅ Final merged dataset saved:", out_path)
print("✅ Shape:", merged_df.shape)


--- Before Dropping Sparse Columns ---
Shape: 3613 rows × 46 cols
                                           Missing_Count  Missing_Percent
Reconstruction Costs ('000 US$)                     3606        99.806255
Reconstruction Costs, Adjusted ('000 US$)           3606        99.806255
AID Contribution ('000 US$)                         3522        97.481317
River Basin                                         3194        88.402989
No. Homeless                                        3146        87.074453
Insured Damage, Adjusted ('000 US$)                 3098        85.745918
Insured Damage ('000 US$)                           3095        85.662884
Event Name                                          3013        83.393302
Latitude                                            2944        81.483532
Longitude                                           2944        81.483532
External IDs                                        2481        68.668696
No. Injured                                  